In [1]:
import findspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import window
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, FloatType, StringType


In [2]:
findspark.init('/opt/spark-3.5.0/')

spark = SparkSession\
    .builder\
    .appName("InteractionCount")\
    .config("spark.eventLog.enabled", "true")\
    .config("spark.eventLog.dir", "applicationHistory")\
    .master("local[*]")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/25 21:06:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/25 21:06:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
interactions_schema = StructType() \
    .add(StructField('user_a', StringType())) \
    .add(StructField('user_b', StringType())) \
    .add(StructField('timestamp', TimestampType())) \
    .add(StructField('interaction', StringType()))

In [4]:
interactions = spark.readStream.schema(interactions_schema).option('sep', ',').csv('interactions/*.csv')

In [5]:
windowed_count = interactions \
    .withWatermark(eventTime='timestamp', delayThreshold='5 minute') \
    .groupBy(window(timeColumn='timestamp', windowDuration='1 minute', slideDuration='40 seconds'), "user_a") \
    .count() \
    .orderBy('window')


In [ ]:
 
windowed_count.writeStream \
    .format('console') \
    .outputMode('complete') \
    .option('truncate', 'false') \
    .option('numRows', '10000') \
    .trigger(processingTime='12 seconds') \
    .start() \
    .awaitTermination()


24/03/25 21:06:31 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/v4/r34w0mtd5xzfzyxp6hnbp8y00000gn/T/temporary-d3b35c37-64b7-45a8-b193-867b695f2887. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/25 21:06:31 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+------+-----+
|window                                    |user_a|count|
+------------------------------------------+------+-----+
|{2012-07-04 00:56:00, 2012-07-04 00:57:00}|454613|4    |
|{2012-07-04 00:56:40, 2012-07-04 00:57:40}|454613|4    |
|{2012-07-05 00:56:00, 2012-07-05 00:57:00}|454613|3    |
|{2012-07-05 00:56:40, 2012-07-05 00:57:40}|454613|3    |
|{2012-07-06 00:56:00, 2012-07-06 00:57:00}|454613|1    |
|{2012-07-06 00:56:40, 2012-07-06 00:57:40}|454613|1    |
+------------------------------------------+------+-----+



24/03/25 21:06:45 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 12000 milliseconds, but spent 13985 milliseconds
